<a href="https://colab.research.google.com/github/rajkumarshedage/Health-Insurance-Cross-Sell-Prediction/blob/main/Health_Insurance_Cross_Sell_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Name - Health Insurance Cross Sell Prediction
## Project Type - Classification
## Contribution - Individual
### GitHub Link
https://github.com/rajkumarshedage/Health-Insurance-Cross-Sell-Prediction

### Problem Statement
#### Business Problem Overview
An insurance company is looking to build a model to predict whether their past health insurance policyholders would also be interested in purchasing vehicle insurance from the company.

An insurance policy is a contract where the company agrees to provide compensation for specific losses, injuries, or death in exchange for regular payments called premiums.

In this case, the company wants to use information about the customer's demographics, vehicle details, and previous insurance policy information to predict their interest in vehicle insurance.

This information would be used to target marketing efforts and optimize the company's revenue.

## Business Objective
The business objective for a health insurance cross-sell prediction is to identify potential customers who are likely to purchase additional products.

This allows the insurance company to target their marketing efforts towards these individuals, potentially increasing sales and revenue.

Additionally, by identifying and targeting high-risk customers, the insurance company can also improve their risk management and profitability.

In [2]:
## Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.stats as stats
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,classification_report,accuracy_score,roc_auc_score,roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

In [3]:
## Dataset Loading

data = pd.read_csv('/content/drive/MyDrive/Projects_DS/Health Insurance Cross Sell Prediction/TRAIN-HEALTH INSURANCE CROSS SELL PREDICTION.csv')

In [4]:
## Viewing data's first 5 row

data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [5]:
# Checking shape of data

data.shape

(381109, 12)

### Data has 381109 rows and 12 columns.
### Variables Description
id : Unique ID for customer

Gender : Male/Female

Age : Age of customer

Driving License : Customer has DL or not

Region_Code : Unique code for the region of the customer

Previously_insured : Customer already has vehicle insurance or not

Vehicle_age : Age of the vehicle

Vehicle_damage : Past damages present or not

Annual_premium : The amount customer needs to pay as premium

PolicySalesChannel : Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc

Vintage : Number of Days, Customer has been associated with the company

Response : Customer is interested or not

In [6]:
## Checking data types

data.dtypes

id                        int64
Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
Response                  int64
dtype: object

## Chechking null or missing values

In [7]:
data.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

### In our data there no null or missing values

## Checking unique values in each feature

In [8]:
data.nunique()

id                      381109
Gender                       2
Age                         66
Driving_License              2
Region_Code                 53
Previously_Insured           2
Vehicle_Age                  3
Vehicle_Damage               2
Annual_Premium           48838
Policy_Sales_Channel       155
Vintage                    290
Response                     2
dtype: int64

## Data Describe

In [9]:
data.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,190555.000000,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,381109.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [10]:
## Creating copy of the current data and assigning to df

df = data.copy()

In [11]:
## Creating interested Response dataset

df_Response = df[(df['Response'] == 1)]

In [12]:
df['Response'].value_counts()

0    334399
1     46710
Name: Response, dtype: int64

## Exploring Response Labels - Univariate analysis